## 7_building_height_calculation_parquet_ver
### Adds the building height to all buildings in the curated labelled building catalog
### Building height is not used by the current model as an input for classification, hence it is not used in the training process, because building heights have not improved the models precision. The code itself is retained, nevertheless.

### Initial configuration
#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "COS_ENDPOINT_URL": "s3.private.eu-de.cloud-object-storage.appdomain.cloud",
    "COS_AUTH_ENDPOINT_URL": "https://iam.cloud.ibm.com/oidc/token",
    "COS_APIKEY": "xxx",
    "DATA_CURATION_BUCKET": "xxx",
    "UTILS_BUCKET": "notebook-utils-bucket",
    "HEIGHTS_TIFF_FILENAME": "WSF3Dv3_Kenya.tif",
    "BUCKET_TIFF": "buildings-height-tiffs"
    }
    """


In [22]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [24]:
# Import necessary libraries
import io
from PIL import Image
import ibm_boto3
import jaydebeapi as jdbc
import jpype
from botocore.client import Config
import numpy as np
import configparser
import os
import sys
from ibm_cloud_sdk_core import ApiException
from ibmcloudant.cloudant_v1 import CloudantV1, Document, BulkDocs
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pandas as pd
import geopandas as gpd
import random
import time
import base64
import shutil
import threading
from collections import Counter
from tqdm import tqdm
from datetime import datetime
import os
import rasterio
from rasterio.windows import Window
import gc
#import cv2
from matplotlib.path import Path
import matplotlib.pyplot as plt
import shapely


# # init S3 client in order to work with last tiff file version
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config["COS_APIKEY"],
                              ibm_auth_endpoint=config["COS_AUTH_ENDPOINT_URL"],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config["COS_ENDPOINT_URL"])


# import external utils library
response = cos_client.list_objects_v2(Bucket=config["UTILS_BUCKET"])

try:
    for obj in response['Contents']:
        name = obj['Key']
        streaming_body_1 = cos_client.get_object(Bucket=config["UTILS_BUCKET"], Key=name)['Body']
        print("Copying to localStorage :  " + name)
        with io.FileIO(name, 'w') as file:
            for i in io.BytesIO(streaming_body_1.read()):
                file.write(i)
    
    from utils import *
    print('External utils succesfully imported')
except Exception as e:
    print('Error occured: ', e)
    

Copying to localStorage :  Kenya_urban_suburban.json
Copying to localStorage :  db2jcc4.jar
Copying to localStorage :  utils.py
External utils succesfully imported


In [26]:
# assign cinfig necessary variables
BUCKET_TIFF = config["BUCKET_TIFF"]
heights_tiff_name = config["HEIGHTS_TIFF_FILENAME"]
labelled_data_SMOD_parquet = 'all_labelled_data_SMOD.parquet'
labelled_data_SMOD_heights_parquet = 'all_labelled_data_SMOD_heights.parquet'
curation_bucket = config["DATA_CURATION_BUCKET"]

In [27]:
# paths for local storing of tiff files
path_to_tif_folder = 'tiff/'

# clear files in directories if exist
try:
    shutil.rmtree(path_to_tif_folder, ignore_errors=True)
except Exception as e:
    print(e)

print("Recreate /tiff directories")
os.makedirs(os.path.dirname(path_to_tif_folder), exist_ok=True)

# download heights tiff file
streaming_body = cos_client.get_object(Bucket=BUCKET_TIFF, Key=heights_tiff_name)['Body']      
with io.FileIO(path_to_tif_folder + heights_tiff_name, 'w') as file:
    print("Copying to localStorage: " + path_to_tif_folder + heights_tiff_name)
    for i in io.BytesIO(streaming_body.read()):
        file.write(i)

print('Successfully downloaded')

Recreate /tiff directories


Copying to localStorage: tiff/WSF3Dv3_Kenya.tif
Successfully downloaded


In [28]:
#open heights tiff 
dat = rasterio.open(os.path.join(path_to_tif_folder, heights_tiff_name))
profile = dat.profile.copy()
profile.update(compress='lzw')
print(profile)

#divide tiff to tiles
tiff_width = profile['width']
tiff_height = profile['height']

tile_width = int(tiff_width / 10)
tile_height = int(tiff_height / 10)

print(f'tile_width: {tile_width}, tile_height: {tile_height}')
# define overlap between tiles
overlap = 500

columns_amount = int(tiff_width / tile_width) if tiff_width % tile_width == 0 else int(tiff_width / tile_width) + 1
rows_amount = int(tiff_height / tile_height) if tiff_height % tile_height == 0 else int(tiff_height / tile_height) + 1
print(f'TIFf image wiil be divided to {rows_amount} rows and {columns_amount} cols')

images_coords = []

for col_idx in range(1, columns_amount + 1):
    
    row_start = max(tile_width * (col_idx - 1) - overlap, 0)
    
    if col_idx != columns_amount:
        
        row_limits = [row_start, tile_width * col_idx]
    elif col_idx == columns_amount:
        row_limits = [row_start, tiff_width]
    
    for row_idx in range(1, rows_amount + 1):
        
        col_start = max(tile_height * (row_idx - 1) - overlap, 0)
        
        if row_idx != columns_amount:
            col_limits = [col_start, tile_height * row_idx]
        elif row_idx == columns_amount:
            col_limits = [col_start, tiff_height]
            
        coords = [col_limits, row_limits]
        images_coords.append(coords)
        
print(len(images_coords))

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 111543, 'height': 133808, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(8.983152841195211e-05, 0.0, 31.98999541430869,
       0.0, -8.983152841195211e-05, 6.010088576873247), 'blockysize': 1, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
tile_width: 11154, tile_height: 13380
TIFf image wiil be divided to 11 rows and 11 cols
121


In [29]:
# Fetch the labelled data set and the SMOD polygons
if type(curation_bucket) == str:

    streaming_body = cos_client.get_object(Bucket=curation_bucket, Key=labelled_data_SMOD_parquet)['Body']
    print("Downloading to local storage :  " + labelled_data_SMOD_parquet)
    with io.FileIO(labelled_data_SMOD_parquet, 'w') as file:
        for i in io.BytesIO(streaming_body.read()):
            file.write(i)

buildings_no_height = pd.read_parquet(labelled_data_SMOD_parquet)
buildings_no_height = gpd.GeoDataFrame(buildings_no_height, geometry=shapely.from_wkb(buildings_no_height.geometry))
buildings_no_height

,latitude,longitude,vida_confidence,geometry,building_area_in_meters,county,SMOD_name,SMOD_id
5588,-0.883311,35.014648,0.7856,"POLYGON ((35.01468 -0.88330, 35.01463 -0.88329...",21.586176,Bomet,Dense and semi-dense urban cluster (Town),5
5606,-0.882994,35.014654,0.0000,"POLYGON ((35.01468 -0.88302, 35.01468 -0.88298...",25.149882,Bomet,Dense and semi-dense urban cluster (Town),5
5614,-0.883239,35.014658,0.8072,"POLYGON ((35.01468 -0.88326, 35.01468 -0.88321...",24.440989,Bomet,Suburban or peri-urban cells (Suburb),4
6765,-0.882690,35.015054,0.7530,"POLYGON ((35.01507 -0.88271, 35.01507 -0.88267...",22.086248,Bomet,Suburban or peri-urban cells (Suburb),4
7126,-0.883227,35.015184,0.8424,"POLYGON ((35.01521 -0.88320, 35.01516 -0.88320...",35.753080,Bomet,Suburban or peri-urban cells (Suburb),4
...,...,...,...,...,...,...,...,...
1024172,-1.263098,37.096335,0.8918,"POLYGON ((37.09638 -1.26312, 37.09637 -1.26310...",81.492523,Nairobi,Low density rural grids cells (Dispersed rural...,2
1024188,-1.266434,37.101554,0.8751,"POLYGON ((37.10160 -1.26645, 37.10158 -1.26639...",72.363677,Nairobi,Low density rural grids cells (Dispersed rural...,2
1024189,-1.268861,37.101554,0.8099,"POLYGON ((37.10157 -1.26883, 37.10155 -1.26885...",41.212594,Nairobi,Low density rural grids cells (Dispersed rural...,2
1024217,-1.265807,37.101614,0.8286,"POLYGON ((37.10165 -1.26583, 37.10164 -1.26577...",45.291012,Nairobi,Low density rural grids cells (Dispersed rural...,2


## Loop through all tiles and calculate heights stats for all available buildings in tile

In [ ]:
# treshold for area
threshold = 0

# cursor = DB2_connection.cursor()
dfs = []
#images_coords = [[[53020, 66900], [10654, 22308]]]
t1 = time.time()

# loop through tiles coords
for idx, coords in enumerate(images_coords):
    init_time = time.time()
    with rasterio.open(os.path.join(path_to_tif_folder, heights_tiff_name)) as src:
        
        # read tiff metadata by coords in order to prepare filtered dataframe
        print(coords)
        
        col_off = coords[1][0]
        row_off = coords[0][0]
        
        width = coords[1][1] - coords[1][0]
        height = coords[0][1] - coords[0][0]
        
        # tiff_data = src.read(1, window=Window(col_off, row_off, width, height))
        
        lon_upper_left, lat_upper_left = src.xy(coords[0][0], coords[1][0])
        lon_down_right, lat_down_right = src.xy(coords[0][1], coords[1][1])

        lons_sorted = sorted([lon_upper_left, lon_down_right])
        lats_sorted = sorted([lat_upper_left, lat_down_right])
        
        lon_min = lons_sorted[0]
        lon_max = lons_sorted[1]

        lat_min = lats_sorted[0]
        lat_max = lats_sorted[1]
        
        areas_covered_by_tifs = create_bounds_dict(path_to_tifs=path_to_tif_folder)

        df = buildings_no_height[
                (buildings_no_height.latitude >= lat_min) & \
                (buildings_no_height.latitude <= lat_max) & \
                (buildings_no_height.longitude >= lon_min) & \
                (buildings_no_height.longitude <= lon_max)
            ].copy()
        
        
        if len(df) > 0:

            # coordinates of current tile
            col_off = max(coords[1][0] - 1, 0)
            row_off = max(coords[0][0] - 1, 0)
            width = min(coords[1][1] - coords[1][0] + 1, tiff_width)
            height = min(coords[0][1] - coords[0][0] + 1, tiff_height)
            
            # read tile grayscale layer
            tiff_data = src.read(1, window=Window(col_off, row_off, width, height))
            print(f"Images revealed: {len(df)}")
            
            # loop through building centroids inside tile
            for index, row, in tqdm(df.iterrows(), total=len(df), desc='Height calculation'):
                try:

                    # lat lon to pixel transformations
                    pixel_coordinates = get_pixel_coordinates(row.geometry, areas_covered_by_tifs, src)
                    polygon_coordinates = [[pixel_coords[0] - row_off, pixel_coords[1] - col_off] for pixel_coords in pixel_coordinates]
                    
                    margin = 0
                    
                    rowcolminmax = get_min_max_values_of_row_col(pixel_coordinates=polygon_coordinates)
                    
                    img_width = rowcolminmax['rowminmax'][1] - rowcolminmax['rowminmax'][0] 
                    img_height = rowcolminmax['colminmax'][1] - rowcolminmax['colminmax'][0]
                    
                    row_start = rowcolminmax['rowminmax'][0]
                    row_end = rowcolminmax['rowminmax'][1]
                    col_start = rowcolminmax['colminmax'][0]
                    col_end = rowcolminmax['colminmax'][1]
                    img_array_pre = np.array(tiff_data[row_start : row_end, col_start : col_end])
                    

                    polygon_coordinates = offset_polygon_coords(polygon_coordinates)
                    rowcolminmax = get_min_max_values_of_row_col(pixel_coordinates=polygon_coordinates)

                    img_width = rowcolminmax['rowminmax'][1] - rowcolminmax['rowminmax'][0] 
                    img_height = rowcolminmax['colminmax'][1] - rowcolminmax['colminmax'][0]
                    row_start = rowcolminmax['rowminmax'][0]
                    row_end = rowcolminmax['rowminmax'][1]
                    col_start = rowcolminmax['colminmax'][0]
                    col_end = rowcolminmax['colminmax'][1]

                    # cut building image from tile
                    img_array_pre = np.array(tiff_data[row_start : row_end, col_start : col_end])
                    
                    # extract building by polygon coords
                    absolule_polygon_coordinates = [[pixel_coords[0] - row_start, pixel_coords[1] - col_start] for pixel_coords in polygon_coordinates]
                    poly_path=Path(absolule_polygon_coordinates)
                    x, y = np.mgrid[:img_height, :img_width]
                    coors = np.hstack((x.reshape(-1, 1), y.reshape(-1,1)))
                    mask = poly_path.contains_points(coors).reshape(img_height, img_width).T
                    
                    # create zeros mask
                    img_masked=np.zeros((img_width, img_height),dtype=img_array_pre.dtype)

                    # put image on zeros mask
                    img_masked[mask]=img_array_pre[mask]

                    # extract image as list of non zero values
                    img_masked_list = list(filter(lambda num: num != 0, img_masked.flatten(order='C')))

                    #calculate height statistics
                    df.at[index, 'height_mean_by_poly'] = 0 if len(img_masked_list) == 0 else np.mean(img_masked_list)
                    df.at[index, 'height_median_by_poly'] = 0 if len(img_masked_list) == 0 else np.median(img_masked_list)
                    df.at[index, 'height_max_by_poly'] = 0 if len(img_masked_list) == 0 else np.max(img_masked_list)
                    df.at[index, 'height_categorized'] = 3 if len(img_masked_list) == 0 else np.median(img_masked_list) // 3 * 3 + 3


                except Exception as e:
                    pass
                    # print(f'Height error occured: {e}')

            try:
                df = df[df.height_categorized > 0]            
                print(f'Buildings heights calculated: {len(df)}')

                dfs.append(df)
            
            except Exception as e:
                pass
            #Update changed height values        
            # for row in tqdm(df.itertuples(), total=len(df), desc='ingestion_data'):
            #     lon, lat = row.doc_id.split(':')
            #     try:
            #         # upd_height_db2(lat, lon, row.height_categorized, row.height_max_by_poly, row.height_mean_by_poly, row.height_median_by_poly, cursor)
            #     except Exception as e:
            #         print(f"Error of database: {e}")
            #         DB2_connection = connect_to_db()
            #         cursor = DB2_connection.cursor()
            #         # upd_height_db2(lat, lon, row.height_categorized, row.height_max_by_poly, row.height_mean_by_poly, row.height_median_by_poly, cursor)
            
            print(f'Image tile processed, time took: {time.strftime("%H:%M:%S", time.gmtime(int(time.time() - init_time)))}')
    #         if idx == 16:
    #             break
            
            
        
print("")
print(f'All tiles processed, time took: {time.strftime("%H:%M:%S", time.gmtime(int(time.time() - t1)))}')


In [33]:
main_df = pd.concat(dfs)

In [35]:
main_df = main_df.drop_duplicates(subset='geometry')

In [38]:
main_df.to_parquet(labelled_data_SMOD_heights_parquet)

# optionaly upload file to the bucket
if type(curation_bucket) == str:
        
    try:
        cos_client.upload_file(
            Filename=labelled_data_SMOD_heights_parquet,
            Bucket=curation_bucket,
            Key=labelled_data_SMOD_heights_parquet,
            ExtraArgs={'ContentDisposition': 'attachment'}
        )
           
        print(f'File {labelled_data_SMOD_heights_parquet} successfully uploaded to the COS {curation_bucket} bucket')
    except Exception as e:
        print(f"\033[91mFailed upload file to the bucket {curation_bucket}. Error: {e}")